# Power BI Connection - Interactive Authentication (DCF)
Uses **browser-based authentication to support MFA**.

Please note that within a production setting, this **shouldn't be your preferred methodology** of choice as it includes interactive authentication, which is not very scalable. We recommend using a **service-principal-based** approach, which is illustrated in another notebook.

In [0]:
# Client and tenant configuration (should be constant, but can be ingested as job parameters)
TENANT_ID = "9f37a392-f0ae-4280-9796-f1864a10effc"  # Your tenant ID
CLIENT_ID = "1950a258-227b-4e31-a9cf-717495945fc2"  # Power BI public client

In [0]:
%pip install azure-identity requests pandas

In [0]:
# Import the libs
import json
import time
import requests
import sys
import re
import math
import pandas as pd

from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from azure.identity import InteractiveBrowserCredential, DeviceCodeCredential

In [0]:
# Configuration of the job (fetching of the DAX query input parameter)
try:
    job_params = json.loads(dbutils.widgets.get("job_params"))
    DAX_QUERY = job_params.get("dax_statement")
    WORKSPACE_ID = job_params.get("workspace_id")  # Your PowerBI workspace ID
    SEMANTIC_MODEL_ID = job_params.get("semantic_model_id")  # Your semantic-model ID you want to parse
except:
    print("No DAX statement to extract")

# Demonstrate if the ingestion works of the DAX statement
print(f"My DAX statement is: {DAX_QUERY}")
print(f"My WORKSPACE_ID is: {WORKSPACE_ID}")
print(f"My SEMANTIC_MODEL_ID is: {SEMANTIC_MODEL_ID}")

## Device Code Flow authentication

In [0]:
# Implement a function to generate a device control-flow based authentication token (aka manual input will be needed here, thus not production ready and recommended to go via SVPs instead of this flow)
def generate_token_device_code(tenant_id: str, client_id: str) -> str:
    """
    Generate token using device code flow.
    You'll get a code to enter at microsoft.com/devicelogin
    """
    try:
        credential = DeviceCodeCredential(
            client_id=client_id,
            tenant_id=tenant_id,
        )
        
        # Get token for Power BI API
        print("\n⚠️  Follow the instructions above to authenticate")
        token = credential.get_token("https://analysis.windows.net/powerbi/api/.default")
        print("✓ Token generated successfully")
        return token.token
    except Exception as e:
        print(f"✗ Token generation failed: {str(e)}")
        raise

# Uncomment to use device code flow instead
access_token = generate_token_device_code(TENANT_ID, CLIENT_ID)
print(f"Token length: {len(access_token)} characters")

In [0]:
# Function to test token functionality by extracting dataset metadata
def get_dataset_info(token: str, dataset_id: str) -> dict:
    """
    Get metadata about the dataset to verify connection.
    """
    url = f"https://api.powerbi.com/v1.0/myorg/datasets/{dataset_id}"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    response = requests.get(url, headers=headers)
    print(f"Response status: {response.status_code}")
    
    if response.status_code == 200:
        print("✓ Successfully connected to dataset")
        return response.json()
    else:
        print(f"✗ Failed to connect: {response.text}")
        return {}

In [0]:
# Function to execute an input DAX-query against a given PowerBI dataset
def execute_dax_query(token: str, dataset_id: str, dax_query: str) -> pd.DataFrame:
    """
    Execute a DAX query against the Power BI dataset.
    """
    url = f"https://api.powerbi.com/v1.0/myorg/datasets/{dataset_id}/executeQueries"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    body = {
        "queries": [
            {
                "query": dax_query
            }
        ],
        "serializerSettings": {
            "includeNulls": True
        }
    }
    
    print(f"Executing DAX query...")
    print(f"Query: {dax_query}...")
    response = requests.post(url, headers=headers, json=body, timeout=30)
    print(f"Response status: {response.status_code}")
    
    if response.status_code == 200:
        results = response.json().get("results", [])
        if results and results[0].get("tables"):
            rows = results[0]["tables"][0].get("rows", [])
            if rows:
                df = pd.DataFrame(rows)
                print(f"✓ Query successful: {len(df)} rows returned")
                return df
            else:
                print("⚠ Query returned no rows")
                return pd.DataFrame()
        else:
            print("⚠ No tables in response")
            return pd.DataFrame()
    else:
        print(f"✗ Query failed: {response.text}")
        return pd.DataFrame()

## Execution of the code

In [0]:
# Get dataset info
dataset_info = get_dataset_info(access_token, SEMANTIC_MODEL_ID)
if dataset_info:
    print(f"\nDataset Name: {dataset_info.get('name', 'N/A')}")
    print(f"Dataset ID: {dataset_info.get('id', 'N/A')}")
    print(f"Is Refreshable: {dataset_info.get('isRefreshable', 'N/A')}")

In [0]:
# Cleanup the result and test it
df_result = execute_dax_query(access_token, SEMANTIC_MODEL_ID, DAX_QUERY)
if not df_result.empty:
    print("\nQuery Results:")
    display(df_result)
    print(f"\nTotal rows: {len(df_result)}")
    print(f"Columns: {list(df_result.columns)}")